## 4. Diving deeper into Ray Tune concepts

- How does the Tuner **which resources to allocate** to trials?
    - Each trial will run in a separate process and consume 1 CPU core.
- How does it decide **how to tune** - i.e. which trials to run next?
    - Ray Tune uses a search algorithm to decide which trials to run next.
    - e.g. A random search, or a more sophisticated search algorithm like a bayesian optimization algorithm.
- How does it decide **when to stop** - i.e. whether to kill a trial early?
    - e.g. If a trial is performing poorly compared to other trials, it perhaps makes sense to stop it early (successive halving, hyperband)
    - Ray Tune uses a scheduler to decide if/when to stop trials, or to prioritize certain trials over others.

Below is a diagram showing the relationship between the different Ray Tune components we have discussed.

<img src="https://docs.ray.io/en/latest/_images/tune_flow.png" width="800" />


To learn more about the key tune concepts, you can visit the [Ray Tune documentation here](https://docs.ray.io/en/master/tune/key-concepts.html).

### Default settings for Ray Tune

In [ ]:
tuner = tune.Tuner(
    # This is how to specify resources for your trainable function
    trainable=tune.with_resources(train_my_simple_model, {"cpu": 1}),
    param_space={"a": tune.randint(0, 20)},
    tune_config=tune.TuneConfig(
        mode="min",
        metric="rmse",
        num_samples=5, 
        # This search algorithm is a basic variation (i.e random/grid search) based on parameter space
        search_alg=tune.search.BasicVariantGenerator(), 
        # This scheduler is very simple: no early stopping, just run all trials in submission order
        scheduler=tune.schedulers.FIFOScheduler(), 
    ),
)
results = tuner.fit()

Below is a diagram showing the relationship between the different Ray Tune components we have discussed.

<img src="https://docs.ray.io/en/latest/_images/tune_flow.png" width="800" />


To learn more about the key tune concepts, you can visit the [Ray Tune documentation here](https://docs.ray.io/en/master/tune/key-concepts.html).

### Annotated experiment table

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/attentive-ray-101/experiment_table_annotated.png" width="800" />




#### Exercise

<div class="alert alert-block alert-info">
    
__Lab activity: Finetune a linear regression model.__
    

Given the below code to train a linear regression model from scratch: 

```python
def train_linear_model(lr: float, epochs: int) -> None:
    x = np.array([0, 1, 2, 3, 4])
    y = x * 2
    w = 0
    for _ in range(epochs):
        loss = np.sqrt(np.mean((w * x - y) ** 2))
        dl_dw = np.mean(2 * x * (w * x - y)) 
        w -= lr * dl_dw
        print({"rmse": loss})

# Hint: Step 1 update the function signature

# Hint: Step 2 Create the tuner object
tuner = tune.Tuner(...)

# Hint: Step 3: Run the tuner
results = tuner.fit()
```

Use Ray Tune to tune the hyperparameters `lr` and `epochs`. 

Perform a search using the optuna.OptunaSearch search algorithm with 5 samples over the following ranges:
- `lr`: loguniform(1e-4, 1e-1)
- `epochs`: randint(1, 100)

</div>


In [ ]:
# Write your code here


<div class="alert alert-block alert-info">
<details>
<summary>Click here to view the solution</summary>

```python
def train_linear_model(config) -> None:
    epochs = config["epochs"]
    lr = config["lr"]
    x = np.array([0, 1, 2, 3, 4])
    y = x * 2
    w = 0
    for _ in range(epochs):
        loss = np.sqrt(np.mean((w * x - y) ** 2))
        dl_dw = np.mean(2 * x * (w * x - y)) 
        w -= lr * dl_dw
        train.report({"rmse": loss})

tuner = tune.Tuner(
    trainable=train_linear_model,  # Training function or class to be tuned
    param_space={
        "lr": tune.loguniform(1e-4, 1e-1),  # Hyperparameter: learning rate
        "epochs": tune.randint(1, 100),  # Hyperparameter: number of epochs
    },
    tune_config=tune.TuneConfig(
        metric="rmse",  # Metric to optimize (minimize)
        mode="min",     # Minimize the metric
        num_samples=5,  # Number of samples to try
        search_alg=optuna.OptunaSearch(), # Use Optuna for hyperparameter search
    ),
)

results = tuner.fit()
```

</details>
</div>